In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Train dataset is read and the rows with missing target value is removed from the dataset.

In [ ]:
X_train = pd.read_csv('/kaggle/input/debt-default-prediction/train.csv')
X_train.dropna(axis=0, subset=['loan_status'], inplace=True)
X_train.head()
y_train = X_train.loan_status
X_train.drop(['loan_status'], axis = 1,inplace= True )


In [ ]:
print(X_train.shape)
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column >0])

In [ ]:
num_cols_with_missing = (missing_val_count_by_column >100000).sum()
num_cols_with_missing

In [ ]:
# Identify columns with more than 50,000 missing values
cols_to_drop = missing_val_count_by_column[missing_val_count_by_column > 100000].index

# Ensure that the columns to drop actually exist in X_train
cols_to_drop = [col for col in cols_to_drop if col in X_train.columns]

# Drop columns from X_train
X_train.drop(cols_to_drop, axis=1, inplace=True)

# Print the shape of X_train after dropping columns
print(X_train.shape)

In [ ]:
X_valid = pd.read_csv('/kaggle/input/debt-default-prediction/valid.csv')
X_valid.dropna(axis=0, subset=['loan_status'], inplace=True)

y_valid = X_valid.loan_status
X_valid.drop(['loan_status'], axis = 1,inplace= True )
train_columns  = X_train.columns
X_valid = X_valid[train_columns]
X_valid.shape

* **Let us handle the rest of the missing values with imputations , numerical values with the mean of the column and the object columns with the most frequent value.**

In [ ]:
unique_values_counts = X_train.nunique()

# Find columns where number of unique values is equal to 1
columns_with_same_value = unique_values_counts[unique_values_counts == 1].index.tolist()

# Print columns with the same value for every entry

X_train = X_train.drop(columns=columns_with_same_value)
X_valid = X_valid.drop(columns=columns_with_same_value)
print(X_train.shape, X_valid.shape)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]

categorical_cols = [cname for cname in X_train.columns if 
                    X_train[cname].dtype == 'object']

numerical_imputer = SimpleImputer(strategy='mean')
X_train_numerical = X_train[numerical_cols].copy()
X_valid_numerical = X_valid[numerical_cols].copy()
X_train_numerical = numerical_imputer.fit_transform(X_train_numerical)
X_valid_numerical = numerical_imputer.transform(X_valid_numerical)

# Preprocessing for categorical data
categorical_imputer = SimpleImputer(strategy='most_frequent')
X_train_categorical = X_train[categorical_cols].copy()
X_valid_categorical = X_valid[categorical_cols].copy()
X_train_categorical = categorical_imputer.fit_transform(X_train_categorical)
X_valid_categorical = categorical_imputer.transform(X_valid_categorical)

# Convert back to DataFrame

X_train_numerical = pd.DataFrame(X_train_numerical, columns=numerical_cols)
X_train_categorical = pd.DataFrame(X_train_categorical, columns=categorical_cols)
X_valid_numerical = pd.DataFrame(X_valid_numerical, columns=numerical_cols)
X_valid_categorical = pd.DataFrame(X_valid_categorical, columns=categorical_cols)


# Now you can combine the numerical and categorical data
X_train = pd.concat([X_train_numerical, X_train_categorical], axis=1)
X_valid =  pd.concat([X_valid_numerical, X_valid_categorical], axis=1)


In [ ]:
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column >0])
missing_val_count_by_column_valid = (X_valid.isnull().sum())
print(missing_val_count_by_column_valid[missing_val_count_by_column_valid >0])
print(X_train.shape, X_valid.shape)

In [ ]:
X_train[categorical_cols].head()

In [ ]:
X_train['emp_length'].unique()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
#custom_order_grade = ['A', 'B','C','D','E','F','G']
custom_order_subgrade = ['A1','A2','A3','A4','A5','B1','B2','B3','B4','B5','C1','C2','C3','C4','C5','D1','D2','D3','D4','D5','E1','E2','E3','E4','E5','F1','F2','F3','F4','F5','G1','G2','G3','G4','G5']
ordinal_encoder = OrdinalEncoder(categories=[custom_order_subgrade])
X_train['sub_grade'] = ordinal_encoder.fit_transform(X_train[['sub_grade']])


In [ ]:
X_valid['sub_grade'] = ordinal_encoder.transform(X_valid[['sub_grade']]) 
custom_order_emp_length = ['< 1 year', '1 year', '2 years', '3 years','4 years',  '5 years','6 years', '7 years','8 years','9 years','10+ years']
ordinal_encoder2 = OrdinalEncoder(categories=[custom_order_emp_length])
X_train['emp_length'] = ordinal_encoder2.fit_transform(X_train[['emp_length']])
X_valid['emp_length'] = ordinal_encoder2.transform(X_valid[['emp_length']])
print('done')

In [ ]:
X_valid['sub_grade'].head()

In [ ]:

X_train_modified= X_train.drop(columns = ['grade'])
X_valid_modified = X_valid.drop(columns = ['grade']) # redundant feature with grade


In [ ]:
X_valid_modified.shape

In [ ]:
# List to store features with unequal unique values
columns_with_different_unique_values = []

categorical_cols = [cname for cname in X_train_modified.columns if 
                    X_train_modified[cname].dtype == 'object']
#print(categorical_cols)
# Iterate over each feature
for col in categorical_cols:
    # Get unique values of the feature in train and validation sets
    train_unique_values = set(X_train_modified[col].unique())
    valid_unique_values = set(X_valid_modified[col].unique())
    #print(col,X_train[col].nunique() ,X_valid[col].nunique())
    
    # Check if unique values are not equal
    if not valid_unique_values.issubset(train_unique_values):
        columns_with_different_unique_values.append(col)
        print(col, X_train_modified[col].nunique(),X_valid_modified[col].nunique())


# Print features with unequal unique values
print("Features with unequal unique values between X_train and X_valid:")
print(columns_with_different_unique_values)

In [ ]:
good_label_cols = [col for col in categorical_cols if 
                   set(X_valid_modified[col]).issubset(set(X_train_modified[col]))]

bad_label_cols = list(set(categorical_cols)-set(good_label_cols))
        
print('Categorical columns that will be ordinal encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)        

The bad labels are removed since in the valid data set those features contain different labels from the train dataset

In [ ]:
X_train_pre_encoding = X_train_modified.drop(bad_label_cols, axis=1)
X_valid_pre_encoding= X_valid_modified.drop(bad_label_cols, axis=1)

In [ ]:
X_train_pre_encoding.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X_train_encoded = X_train_pre_encoding.copy()
X_valid_encoded = X_valid_pre_encoding.copy()
for column in good_label_cols:  # Iterate over categorical columns
    X_train_encoded[column] = label_encoder.fit_transform(X_train_pre_encoding[column])
for column in good_label_cols:  # Iterate over categorical columns
    X_valid_encoded[column] = label_encoder.transform(X_valid_pre_encoding[column])
